In [ ]:
#import MEDYANSimRunner
using MEDYAN
using MEDYANVis
using StaticArrays
using LinearAlgebra
using Random
using Setfield
using OrderedCollections: OrderedDict
using SmallZarrGroups
using CairoMakie
using ForwardDiff

In [ ]:
L = 4

In [ ]:
grid = CubicGrid(SA[L,L,L], 500.0)

In [ ]:
c = MEDYAN.example_actin_mech_context(grid)

In [ ]:
num_monomers = 10

In [ ]:
monomer_len = 2.7 #nm

In [ ]:
c.sys_def.state.actin

In [ ]:
filament_points = [
    SA[-num_monomers*monomer_len/2, 0, 0],
    SA[+num_monomers*monomer_len/2, 0, 0],
]

In [ ]:
chem_newfilament!(c;
    ftid=1, 
    monomerstates=ones(UInt8,num_monomers), 
    node_mids=[0,], 
    nodepositions=filament_points,
)

In [ ]:
MEDYAN.ForceContext(c::MEDYAN.Context)

In [ ]:
function Hessian(fc,x0;dx=0.0001)
    d = 0.1 # 1Å displacement
    MEDYAN.refresh_neighborlists!(fc,x0)
    MEDYAN.calc_all_force_energy!(fc, x0)
    x = copy(x0)
    force0 = copy(fc.forces)
    force_pls = zeros(length(x))
    force_min = zeros(length(x))
    energy = fc.energy[]
    hess = zeros(length(x),length(x))
    for i in 1:length(x)
        x .= x0
        x[i] += dx
        MEDYAN.calc_all_force_energy!(fc, x)
        force_pls .= fc.forces
        x .= x0
        x[i] -= dx
        MEDYAN.calc_all_force_energy!(fc, x)
        force_min .= fc.forces
        hess[:,i] .= (force_min .- force_pls)*inv(2*dx)
    end
    hess = 0.5*(hess + hess')
    return energy,force0,hess
end

function HessianForwardDiff(E,x)
    energy = E(x)
    force = -ForwardDiff.gradient(E,x)
    hess = ForwardDiff.hessian(E,x)
    return energy,force,hess
    #return hess
end

In [ ]:
function energy2points(x;k,L)
    r1= x[1:3]
    r2= x[4:6]
    1//2*k*(norm(r2-r1)-L)^2
end

In [ ]:
config_rand = rand(6)

In [ ]:
display.(HessianForwardDiff(vcat(config_rand...)) do x
    energy2points(x; k= MEDYAN.ACTIN_FIL_PARAMS.klength/num_monomers, L=num_monomers*monomer_len)
    end)

In [ ]:
#display.(Hessian(MEDYAN.ForceContext(c), collect(vcat(filament_points...))))
display.(Hessian(MEDYAN.ForceContext(c), config_rand))

In [ ]:
"Simulation cube length in 500 nm segments."
L = 1

"Number of steps to take."
NSTEPS = 1000

jobs = ["1", "2", "3",]

"""
Return the header_dict and context.
"""
function setup(job::String; kwargs...)
    agentnames = MEDYAN.AgentNames(
        diffusingspeciesnames= [:a,:cl,:m],
        filamentnames= [(:actin,[
                                :plusend,
                                :minusend,
                                :middle,
                                :bound,
                            ]),
        ],
        link_2mon_names= [
            :motor,
            :crosslinker,
        ]
    )
    grid = CubicGrid((L,L,L),500.0)
    monomerspacing = MEDYAN.ACTIN_FIL_PARAMS.spacing
    begin
        s= MEDYAN.SysDef(agentnames)

        add_diffusion_coeff!(s, :a, 20E6)
        add_diffusion_coeff!(s, :cl, 2.0E6)
        add_diffusion_coeff!(s, :m, 0.2E6)

        add_filament_params!(s, 
            :actin,
            MEDYAN.ACTIN_FIL_PARAMS,
        )

        add_link_2mon!(s,
            :motor,
            Link2MonState((numHeads=20,),(L0=NaN,)),
            MEDYAN.DistanceRestraintMechParams(k=55.0),
        )
        
        add_link_2mon!(s,
            :crosslinker,
            Link2MonState((;),(L0=NaN,)),
            MEDYAN.DistanceRestraintMechParams(k=8.0),
        )
        
        #plus end polymerization
        addfilamentend_reaction!(s, :actin, :pp, false,
            [:plusend]=>[:middle,:plusend], monomerspacing,
            "diffusing.a -->", 0.154*500^3, 1,
        )
        #plus end depolymerization
        addfilamentend_reaction!(s, :actin, :dpp, false,
            [:middle,:plusend]=>[:plusend], 0.0,
            "--> diffusing.a", 1.4, 0,
        )

        #minus end polymerization
        addfilamentend_reaction!(s, :actin, :mp, true,
            [:minusend]=>[:minusend,:middle], monomerspacing,
            "diffusing.a -->", 0.0173*500^3, 1,
        )
        #minus end depolymerization
        addfilamentend_reaction!(s, :actin, :dmp, true,
            [:minusend,:middle]=>[:minusend], 0.0,
            "--> diffusing.a", 0.8, 0,
        )

        motorstepsize = 10

        #motor binding
        site = MEDYAN.Decimated2MonSiteMinAngleRange(
            s.filament.actin,
            s.filament.actin,
            motorstepsize,
            motorstepsize,
            s.state.actin.middle,
            s.state.actin.middle,
            175.0,
            225.0,
            cos(5*π/180),
        )
        add_decimated_2mon_site!(s,:motorbinding,site)
        bindcallback = MEDYAN.SimpleMotorBindCallback(
            s.decimated_2mon_site.motorbinding.id,
            s.link_2mon.motor,
            30, #max number of heads
            15, #min number of heads
            s.state.actin.bound,
            [s.diffusing.m=>-1],
        )
        addreactioncallback!(
            s,
            "decimated_2mon_site.motorbinding + diffusing.m",
            0.2*22*500^3/2,
            1,
            bindcallback,
        )

        #motor unbinding
        site = MEDYAN.Link2MonSiteMotorCatch()
        addunbindinglink_2mon_site!(s, 
            :motor, :unbinding, site,
            :actin, :middle, :actin, :middle,
            "--> diffusing.m", 1.0, 0, 
        )

        #motor stepping
        onrate = 0.2
        offrate = 1.7
        dutyratio = onrate/(onrate+offrate)
        site1 = MEDYAN.Link2MonSiteMotorStall(
            fs = 90.0,
            k0 = 6.0/(108/4)*((1 - dutyratio) / dutyratio) * onrate,
            isminusend = true,
        )
        site2 = @set site1.isminusend = false
        add_link_2mon_site!(s,:motor,:motorstepminus,site1)
        add_link_2mon_site!(s,:motor,:motorstepplus,site2)
        stepcallback1 = MEDYAN.SimpleMotorStepCallback(
            lsid = s.link_2mon_site.motor.motorstepminus.id,
            ltid = s.link_2mon.motor,
            unboundstate = s.state.actin.middle,
            boundstate = s.state.actin.bound,
            stepsize = motorstepsize,
        )
        stepcallback2 = @set stepcallback1.lsid = s.link_2mon_site.motor.motorstepplus.id
        addreactioncallback!(
            s,
            "link_2mon_site.motor.motorstepminus",
            1.0,
            0,
            stepcallback1,
        )
        addreactioncallback!(
            s,
            "link_2mon_site.motor.motorstepplus",
            1.0,
            0,
            stepcallback2,
        )

        #crosslinker binding site
        site = MEDYAN.Decimated2MonSiteMinAngleRange(
            s.filament.actin,
            s.filament.actin,
            10,
            10,
            s.state.actin.middle,
            s.state.actin.middle,
            30.0,
            40.0,
            cos(5*π/180)
        )
        add_decimated_2mon_site!(s,:crosslinkbinding,site)
        sitecallback = MEDYAN.SimpleCrosslinkBindCallback(
            s.decimated_2mon_site.crosslinkbinding.id,
            s.link_2mon.crosslinker,
            s.state.actin.bound,
            [s.diffusing.cl=>-1],
        )
        addreactioncallback!(s,
            "decimated_2mon_site.crosslinkbinding + diffusing.cl",
            0.01*500^3/2,
            1,
            sitecallback,
        )

        #crosslinker unbinding
        site = MEDYAN.Link2MonSiteSlipBond(f0 = inv(0.24*MEDYAN.default_β) , k0 = 0.3)
        addunbindinglink_2mon_site!(s, 
            :crosslinker, :unbinding, site,
            :actin, :middle, :actin, :middle,
            "--> diffusing.cl", 1.0, 0, 
        )
    end
    begin
        NMonomers= 40
        monomerstates= zeros(UInt8,NMonomers)
        monomerstates[1:end] .= s.state.actin.middle
        monomerstates[1] = s.state.actin.minusend
        monomerstates[end] = s.state.actin.plusend
    end
    begin
        c= MEDYAN.Context(s,grid;
            g_tol=1.0,
            max_cylinder_force = 1000.0,
            maxstep = 0.7,
        )
        set_mechboundary!(c, MEDYAN.boundary_box(grid; stiffness=100.0))
        adddiffusingcount_rand!(c, s.diffusing.a, 625*L^3)
        adddiffusingcount_rand!(c, s.diffusing.cl, 63*L^3)
        adddiffusingcount_rand!(c, s.diffusing.m, 6*L^3)
        for i in 1:(7*L^3)
            newfilament_rand!(c::MEDYAN.Context, monomerstates)
            MEDYAN.minimize_energy!(c)
        end
    end
    header = OrderedDict([
        "medyan"=>MEDYAN.header(c),
    ])
    return header, c, s
end

"""
Save the context into a group
"""
function save(step::Int, c::MEDYAN.Context; kwargs...)::ZGroup
    group = ZGroup()
    group["medyan"] = MEDYAN.snapshot(c;
        filament_position_scale=5,
        membrane_position_scale=5,
    )
    group
end

"""
Load the context from a group
"""
function load(step::Int, group::ZGroup, c; kwargs...)
    MEDYAN.load_snapshot!(c, group["medyan"]::ZGroup)
    c
end

function done(step::Int, c)
    step ≥ NSTEPS, NSTEPS
end

"""
Move the simulation forward one second in time.
"""
function loop(step::Int, c; kwargs...)
    for j in 1:20
        MEDYAN.run_chemistry!(c,0.05)
        MEDYAN.minimize_energy!(c)
    end
    c
end

In [ ]:
vis = Visualizer()
setvisible!(vis["/Grid"], false)
setvisible!(vis["/Axes"], false)
setvisible!(vis["/Background"], false)

In [ ]:
#set up system and run
nsec = 1
header,c,s = setup(jobs[1])

for i in 1:nsec
    for j in 1:20
        MEDYAN.run_chemistry!(c,0.05)
        #get n-th minimized state
        MEDYAN.minimize_energy!(c)
        
        #calc Hessian
        fc = MEDYAN.ForceContext(c)
        #display.(Hessian(fc,fc.x0;dx=0.0001))
        
        draw_context!(vis, c, s)
        sleep(0.02)
    end
end

In [ ]:
fc = MEDYAN.ForceContext(c)
e,f,h = Hessian(fc,fc.x0;dx=0.0001)

In [ ]:
using SparseArrays

In [ ]:
sparse(h)

In [ ]:
eigvals(h)

In [ ]:
minimum(diag(h))

In [ ]:
h[1:6,1:6]